<a href="https://colab.research.google.com/github/bealu14/AI-Driven-Landslide-Susceptibility-Prediction-for-Early-Risk-Mitigation/blob/main/Spotify_API_DEMO_(2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will fetch and process music data from Spotify.

Specifically we will lookup and display the top tracks for any given artist.



## References



The Spotify API:

  + https://developer.spotify.com/documentation/web-api/
  + https://developer.spotify.com/documentation/general/guides/authorization-guide
  + https://developer.spotify.com/documentation/general/guides/scopes/

The `spotipy` package provides an interface into the Spotify API in Python:

  + https://github.com/plamere/spotipy
  + https://github.com/plamere/spotipy#quick-start
  + https://spotipy.readthedocs.io/en/latest/
  + https://spotipy.readthedocs.io/en/latest/#client-credentials-flow




## Setup



Installing the `spotipy` package into the notebook environment:

In [1]:
%%capture
!pip install spotipy

Verifying package has been installed:

In [ ]:
!pip show spotipy

Name: spotipy
Version: 2.25.1
Summary: A light weight Python library for the Spotify Web API
Home-page: https://spotipy.readthedocs.org/
Author: @plamere
Author-email: paul@echonest.com
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: redis, requests, urllib3
Required-by: 


Create a [Spotify API Client application](https://developer.spotify.com/dashboard/applications/), note its credentials, then set them as notebook secrets called `SPOTIPY_CLIENT_ID` and `SPOTIPY_CLIENT_SECRET` respectively.

Accessing the notebook secrets:

In [ ]:
from google.colab import userdata

SPOTIPY_CLIENT_ID = userdata.get("SPOTIPY_CLIENT_ID")
SPOTIPY_CLIENT_SECRET = userdata.get("SPOTIPY_CLIENT_SECRET")

## Usage

### Info Inputs

Enter your search term:

In [ ]:
#search_term = "Dua Lipa"
#search_term = "Chris Stapleton"
search_term = input("Please input an artist name: ") or "Chainsmokers"
search_term

Please input an artist name: The Weeknd


'The Weeknd'

### Info Processing

Initializing a new Spotify API client, using the provided credentials:

In [ ]:

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

creds = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)

client = Spotify(client_credentials_manager=creds)
print("CLIENT:", type(client))

CLIENT: <class 'spotipy.client.Spotify'>


Fetching tracks for a given artist:

In [ ]:
results = client.search(q=search_term, limit=50)
print(type(results))
print(results.keys())

<class 'dict'>
dict_keys(['tracks'])


In [ ]:
tracks = results["tracks"]["items"]
print(type(tracks))
len(tracks)

<class 'list'>


50

In [ ]:
track = tracks[0]
print(type(track))
track.keys()

<class 'dict'>


dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [ ]:
del track['album']['available_markets']
track['album']

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
   'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
   'id': '1Xyo4u8uXC1ZmMpatF05PJ',
   'name': 'The Weeknd',
   'type': 'artist',
   'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/2ODvWsOgouMbaA5xf0RkJe'},
 'href': 'https://api.spotify.com/v1/albums/2ODvWsOgouMbaA5xf0RkJe',
 'id': '2ODvWsOgouMbaA5xf0RkJe',
 'images': [{'height': 640,
   'width': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2734718e2b124f79258be7bc452'},
  {'height': 300,
   'width': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e024718e2b124f79258be7bc452'},
  {'height': 64,
   'width': 64,
   'url': 'https://i.scdn.co/image/ab67616d000048514718e2b124f79258be7bc452'}],
 'is_playable': True,
 'name': 'Starboy',
 'release_date': '2016-11-25',
 'release_date_precision': 'day',
 'total_tracks': 18,

Parsing and displaying track info:

In [ ]:
info = {
    "name": track['name'],
    "artist": track["artists"][0]["name"],
    #"artists": [artist["name"] for artist in track["artists"]],
    "duration_ms": track['duration_ms'],
    "explicit":  track['explicit'],
    "markets": len(track["available_markets"]),
    "popularity": track["popularity"],
    "album_art_url": track["album"]["images"][0]["url"]
}
info

{'name': 'Starboy',
 'artist': 'The Weeknd',
 'duration_ms': 230453,
 'explicit': True,
 'markets': 184,
 'popularity': 89,
 'album_art_url': 'https://i.scdn.co/image/ab67616d0000b2734718e2b124f79258be7bc452'}

Helper function to fetch and display a table of the artist's tracks:

In [ ]:
from pandas import DataFrame
from IPython.core.display import HTML

def img_html(url, width=50):
    return f"<img src='{url}' width='{width}' >"

# spotify no longer giving previews?
#def preview_html(url):
#    if url:
#        return '<a href="'+ url + '" >Listen on Spotify</a>'
#    else:
#        return None

def display_top_tracks(artist_name:str, top_n=10, client=client):
    results = client.search(q=artist_name, limit=50)

    tracks = results["tracks"]["items"]

    records = []
    for i, track in enumerate(tracks):
        artist_names = [artist["name"] for artist in track["artists"]]
        artist_names = ", ".join(artist_names)
        record = {
            #"index": i,
            #"artist": track["artists"][0]["name"],
            "track": track['name'],
            "artists": artist_names,
            "album": track["album"]["name"],
            "release_date": track["album"]["release_date"],
            #"duration_ms": track["duration_ms"],
            "explicit":  track['explicit'],
            "markets": len(track["available_markets"]),
            "popularity": track["popularity"],
            #"preview_url": preview_html(track["preview_url"]),
            "album_art": img_html(track["album"]["images"][0]["url"])
        }
        records.append(record)

    tracks_df = DataFrame(records)
    # top n:
    tracks_df.sort_values(by="popularity", ascending=False, inplace=True)
    tracks_df = tracks_df.head(top_n)

    tracks_df.reset_index(inplace=True, drop=True)
    tracks_df["rank"] = tracks_df.index + 1
    # reorder cols:
    #cols = ["rank", "name", "artists", "markets", "popularity", "album_art"]
    cols = ["rank"] + tracks_df.drop(columns=["rank"]).columns.tolist()
    tracks_df = tracks_df[cols]

    # displaying the dataframe as HTML:
    tracks_table = HTML(tracks_df.to_html(escape=False, index=False, formatters=dict(Icon=img_html)))
    display(tracks_table)


#display_top_tracks("Ed Sheeran", top_n=10)

### Information Outputs

Displaying top tracks for a given artist:

In [ ]:
display_top_tracks(artist_name=search_term, top_n=10)

rank,track,artists,album,release_date,explicit,markets,popularity,album_art
1,"One Of The Girls (with JENNIE, Lily Rose Depp)","The Weeknd, JENNIE, Lily-Rose Depp",The Idol Episode 4 (Music from the HBO Original Series),2023-06-23,False,183,90,
2,Timeless (feat Playboi Carti),"The Weeknd, Playboi Carti",Hurry Up Tomorrow,2025-01-31,True,183,90,
3,Blinding Lights,The Weeknd,After Hours,2020-03-20,False,184,89,
4,Starboy,"The Weeknd, Daft Punk",Starboy,2016-11-25,True,184,89,
5,The Hills,The Weeknd,Beauty Behind The Madness,2015-08-28,True,184,86,
6,Die For You,The Weeknd,Starboy,2016-11-24,False,184,85,
7,Save Your Tears,The Weeknd,After Hours,2020-03-20,True,184,85,
8,Save Your Tears (Remix) (with Ariana Grande) - Bonus Track,"The Weeknd, Ariana Grande",After Hours (Deluxe),2020-03-20,False,183,82,
9,Creepin' (with The Weeknd & 21 Savage),"Metro Boomin, The Weeknd, 21 Savage",HEROES & VILLAINS,2022-12-02,True,183,81,
10,Die For You,The Weeknd,Starboy,2016-11-25,False,184,79,
